In [2]:
import numpy as np 
import pandas as pd 
import bokeh 
from bokeh.plotting import curdoc, figure, output_file, output_notebook, show
from bokeh.models import ColumnDataSource, ColorMapper, ColorBar, Ticker
from bokeh.transform import linear_cmap, factor_cmap, log_cmap
from bokeh.models.tickers import FixedTicker
from bokeh.models import CustomJS, RadioGroup
from bokeh.layouts import row, column

output_notebook()

Loading BokehJS ...

In [3]:
results = pd.read_csv("data/tess_refined_2g.csv") # this is just the 2g results
triage = pd.read_csv('data/triage_period_table.txt', sep=" ", header=None)
triage.columns = ["TIC", "period", "t0"]
tsne_map = np.load('data/proj_norm.npy')

# prepare data with extra needed columns for plotting
# will also need to insert polyfits as well once I have the table
# results = results.drop('#ind',axis=1)
results.insert(1, "tsne_x", tsne_map[0], True) 
results.insert(2, "tsne_y", tsne_map[1], True) 
results.insert(4, "period_triage", triage['period'], True)

results = results.replace([np.inf, -np.inf], np.nan)
results.insert(5, 'period_reldiff', 100*np.abs(results['period_2g']-results['period_triage'])/results['period_triage'])

In [6]:
results['period_2g'][np.isnan(results['logprob_mean_2g'])] = np.nan*np.ones(len(results['period_2g'][np.isnan(results['logprob_mean_2g'])]))

<ipython-input-6-8e0c4c31402c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['period_2g'][np.isnan(results['logprob_mean_2g'])] = np.nan*np.ones(len(results['period_2g'][np.isnan(results['logprob_mean_2g'])]))


In [7]:
results['period_2g'][np.isnan(results['logprob_mean_2g'])]

1      NaN
2      NaN
3      NaN
8      NaN
21     NaN
        ..
4703   NaN
4712   NaN
4722   NaN
4754   NaN
4767   NaN
Name: period_2g, Length: 699, dtype: float64

In [3]:
df_pf = pd.read_csv('data/tess_refined_polyfits.csv')
df_pf = df_pf.replace([np.inf, -np.inf], np.nan)
df_pf = df_pf[df_pf['logprob_mean_pf'].notnull()]

In [4]:
results = pd.merge(results, df_pf, on=['#ind'], how='outer')

In [5]:
# results = results[~np.isnan(results['logprob_mean_2g'])]
results = results[results['period_reldiff'] <= 200] 

In [6]:
results['period_reldiff'].describe()

count    4768.000000
mean        0.107801
std         1.875301
min         0.000000
25%         0.001146
50%         0.005259
75%         0.020532
max        99.892590
Name: period_reldiff, dtype: float64

In [9]:
source = ColumnDataSource(results)
source_plot = ColumnDataSource({'x': results['period_triage'], 
                                'y': results['period_2g'], 
                                'z': np.zeros(len(results)),
                                'color': np.abs(results['logprob_mean_2g'])})

p0 = figure(plot_width=600, plot_height=400, tools="tap,box_zoom,reset,save,wheel_zoom,hover")
colormapper0 = linear_cmap(field_name = "color", palette='Viridis256', low = 0, high=100)

p0.scatter(x='x', y='y', source = source_plot, color=colormapper0, size=10)
p0.line(x='x', y='x', source = source_plot, line_color='red', line_width=1, line_dash='dotted')
color_bar0 = ColorBar(color_mapper=colormapper0['transform'], width=500, height=10, location=(0,0),
                      title="chi2", orientation="horizontal")
p0.add_layout(color_bar0, 'above')

In [10]:
show(p0)

In [167]:
results_cut = results[results['period_reldiff'] < 1]
def make_plot(title, hist, edges):
    p = figure(plot_width=600, plot_height=400, title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
#     p.y_range.start = 0
#     p.legend.location = "center_right"
#     p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

measured = results_cut['period_reldiff'].values
hist, edges = np.histogram(measured, density=True, bins=50)
p1 = make_plot("|triage-2g|/triage", hist, edges)

In [159]:
show(p1)

In [197]:
4032/4069*100

99.09068567215532

In [230]:
df_pf = pd.read_csv('data/tess_refined_polyfits.csv')

In [231]:
df_pf

,#ind,TIC,period,period_sigma_low,period_sigma_high,logprob_mean,k1,k1_err,k2,k2_err,k3,k3_err,k4,k4_err
0,0,86081744,4.338916,5.492500e-04,2.548000e-05,-1.381212e+03,-0.001394,0.005856,-0.017798,7.490000e-06,0.019540,0.009369,-0.252599,0.009369
1,1,13351941,1.580593,2.700000e-07,3.100000e-07,-2.069980e+06,0.454297,0.000003,-0.454207,2.850000e-06,-0.044788,0.000003,0.045346,0.000003
2,2,79403459,0.435305,1.000000e-07,1.100000e-07,-1.347676e+06,0.066538,0.000011,-0.500000,4.000000e-08,-0.058775,0.000013,0.311482,0.000013
3,3,119461266,2.946297,1.670000e-06,1.670000e-06,-4.029860e+06,0.468023,0.000003,-0.468270,3.400000e-06,-0.032522,0.000004,0.031914,0.000004
4,4,28763683,0.891711,6.000000e-06,7.230000e-06,-2.223554e+03,0.456175,0.001751,-0.403554,4.808800e-04,-0.100123,0.000364,0.090653,0.000364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,4337,399421408,2.045512,6.251000e-05,6.416000e-05,-5.729903e+03,-0.027509,0.000458,-0.479438,5.014900e-04,-0.456743,0.003779,0.027099,0.003779
1424,4338,399577123,0.311978,3.300000e-07,2.800000e-07,-2.229590e+04,0.058028,0.000109,-0.441819,2.144910e-02,-0.115038,0.000905,0.388728,0.000905
1425,4339,399788405,0.430437,3.000000e-08,4.000000e-08,-3.506992e+04,0.416573,0.002560,-0.432756,9.640100e-04,0.074478,0.013013,-0.073490,0.013013
1426,4340,399814051,1.102040,6.000000e-08,3.000000e-08,-5.108035e+05,-0.002798,0.000033,-0.500000,1.500000e-07,-0.060807,0.000110,0.056734,0.000110


In [11]:
df_pf = df_pf.replace([np.inf, -np.inf], np.nan)
df_pf = df_pf[df_pf['logprob_mean_pf'].notnull()]

In [226]:
merged = pd.merge(results, df_pf, on=['#ind'], how='outer')

,#ind,tsne_x,tsne_y,TIC_x,period_triage,period_reldiff,period_2g,period_2g_sigma_low,period_2g_sigma_high,t0_2g,...,period_sigma_high,logprob_mean,k1,k1_err,k2,k2_err,k3,k3_err,k4,k4_err
0,0,-17.534382,22.065203,86081744,4.337107,0.058948,4.339664,9.381000e-04,9.361500e-04,1817.130373,...,2.548000e-05,-1.381212e+03,-0.001394,0.005856,-0.017798,7.490000e-06,0.019540,0.009369,-0.252599,0.009369
1,1,7.185033,-42.929268,13351941,1.580636,0.000093,1.580635,8.600000e-07,8.700000e-07,1791.924808,...,3.100000e-07,-2.069980e+06,0.454297,0.000003,-0.454207,2.850000e-06,-0.044788,0.000003,0.045346,0.000003
2,2,19.460318,-31.579889,79403459,0.435337,0.006197,0.435310,1.500000e-07,1.500000e-07,1325.354083,...,1.100000e-07,-1.347676e+06,0.066538,0.000011,-0.500000,4.000000e-08,-0.058775,0.000013,0.311482,0.000013
3,3,-40.917736,-30.571220,119461266,2.946280,0.002341,2.946211,1.790000e-06,1.850000e-06,1469.949144,...,1.670000e-06,-4.029860e+06,0.468023,0.000003,-0.468270,3.400000e-06,-0.032522,0.000004,0.031914,0.000004
4,4,56.794312,-19.551661,28763683,0.891714,0.001429,0.891701,1.154000e-05,1.107000e-05,1765.320311,...,7.230000e-06,-2.223554e+03,0.456175,0.001751,-0.403554,4.808800e-04,-0.100123,0.000364,0.090653,0.000364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4764,4764,-54.886719,34.026920,394230860,28.965000,0.000301,28.964913,4.290000e-06,4.760000e-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4765,4765,35.535530,28.324120,100508429,2.950000,0.105700,2.953118,1.357000e-05,1.497000e-05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4766,4766,23.242043,14.803665,79328890,0.744088,0.007985,0.744029,3.990000e-06,4.240000e-06,1546.211590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4767,4767,-18.070957,1.761986,375422201,4.075023,0.263423,4.085758,4.372000e-05,2.681000e-05,1741.767635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
results

,#ind,tsne_x,tsne_y,TIC,period_triage,period_reldiff,period_2g,period_2g_sigma_low,period_2g_sigma_high,t0_2g,...,model_2g,logprob_mean_2g,C,mu1,d1,sigma1,mu2,d2,sigma2,Aell
0,0,-17.534382,22.065203,86081744,4.337107,0.058948,4.339664,9.381000e-04,9.361500e-04,1817.130373,...,CG,-0.42,1.000270,-0.001527,0.009002,0.010636,NaN,NaN,NaN,NaN
1,1,7.185033,-42.929268,13351941,1.580636,0.000093,1.580635,8.600000e-07,8.700000e-07,1791.924808,...,CG12E2,NaN,1.021088,0.500000,0.179204,0.024407,0.000045,0.165850,0.024490,0.002505
2,2,19.460318,-31.579889,79403459,0.435337,0.006197,0.435310,1.500000e-07,1.500000e-07,1325.354083,...,CG12E2,NaN,1.030124,0.002908,0.209685,0.036120,-0.500000,0.104874,0.041515,0.014362
3,3,-40.917736,-30.571220,119461266,2.946280,0.002341,2.946211,1.790000e-06,1.850000e-06,1469.949144,...,CG12,NaN,1.039422,0.500000,0.484378,0.016865,0.000380,0.442374,0.017161,NaN
4,4,56.794312,-19.551661,28763683,0.891714,0.001429,0.891701,1.154000e-05,1.107000e-05,1765.320311,...,CG12,-0.42,1.030049,0.498366,0.093212,0.071264,-0.000839,0.082590,0.065360,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4764,4764,-54.886719,34.026920,394230860,28.965000,0.000301,28.964913,4.290000e-06,4.760000e-06,NaN,...,CG12E2,-2.54,1.004612,-0.192744,0.345162,0.002270,0.184906,0.224630,0.004097,0.000195
4765,4765,35.535530,28.324120,100508429,2.950000,0.105700,2.953118,1.357000e-05,1.497000e-05,NaN,...,CG12,-4.45,1.009635,-0.239884,0.012077,0.305768,-0.498939,0.002538,0.062951,NaN
4766,4766,23.242043,14.803665,79328890,0.744088,0.007985,0.744029,3.990000e-06,4.240000e-06,1546.211590,...,CG12E1,-2.21,1.068397,0.266393,0.100930,0.022896,-0.034728,0.099659,0.251121,0.037013
4767,4767,-18.070957,1.761986,375422201,4.075023,0.263423,4.085758,4.372000e-05,2.681000e-05,1741.767635,...,CG12,NaN,1.001438,0.006749,0.041516,0.023900,-0.224082,0.253219,0.000036,NaN


In [1]:
results

NameError: name 'results' is not defined